In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
from bson.json_util import dumps
from pymongo import MongoClient

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
from pandas.core.arrays.sparse import SparseArray as _Sparserray
import pandas as pd
import matplotlib.pyplot as plt



# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
#update with your username and password and CRUD Python module name

username = "aacuser"
password = "password"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
##df.drop(columns=["_id"],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.A([html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                    style={'height' : 200, 'width' : 200,},
                    alt= "Grazioso Salvare Logo. A red drawing of a dog")],
          href='https://www.snhu.edu'),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Footer(html.H5(id='name', children="Mitchell Cabral, CS-340")),
    html.Hr(),
    html.Div(
        
        

        dcc.Dropdown(
        id='filter-dropdown',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Wilderness Rescue'},
            {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster Rescue'},
            {'label': 'All Records', 'value': 'All Records'}
        ],
        clearable=False,
        placeholder="Select to filter / unfilter"
        )
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),

#If you completed the Module Six Assignment, you can copy in the code you created here 
                         filter_action="native",
                         sort_action="native",
                         sort_mode="single",
                         row_selectable="single",
                         selected_rows=[],
                         selected_columns=[],
                         page_current=0,
                         page_size=10,
                        ),
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
              Output('datatable-id','columns')],
              [Input('filter-dropdown', 'value')])
def update_dashboard(value):
    
    if value == 'Water Rescue':
    
        df.pd.DataFrame.from_records(db.read(
        {"animal_type" : "Dog", 
         "breed" : {"$in":["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
         "sex_upon_outcome":"Intact Female",
         "age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}))
    if value == 'Wilderness Rescue':
         df = pd.Dataframe.from_records(db.read(
         {"animal_type": "Dog",
         "breed": {"$in":["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
         "sex_upon_outcome": "Intact Male",
         "age_upon_outcome_in_weeks":{"$gte":26}, "age_upon_outcome_in_weeks":{"$lte":156}}))
    if value == 'Disaster Rescue':
         df = pd.DataFrame.from_records(db.read(
         {"animal_type": "Dog",
         "breed": {"$in":["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
         "sex_upon_outcome": "Intact Male",
         "age_upon_outcome_in_weeks": {"$gte":20}, "age_upon_outcome_in_weeks": {"$lte":300}}))
    else:
         df = pd.DataFrame.from_records(db.read({}))
         
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
         
    return (data,columns)
    
                              


# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
         dff = pd.DataFrame.from_dict(viewData)
         fig=px.pie(dff,
                   names='breed',
                   title="Breeds Available by Rescue Type",
                    hole=.3
                   )
         fig.update_traces(textposition='inside')
         return [
             dcc.Graph(
             id='graph=id',
             figure=fig
             )
         ]
    
    # add code for chart of your choice (e.g. pie chart) #

    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
        Output('map-id', 'children'),
        [Input('datatable-id', 'derived_viewport_data'),
        Input('datatable-id', 'selected_rows')]
)

def update_map(viewData, row):    
    dff = pd.DataFrame.from_dict(viewData)
    #lat = (dff.iloc[row,13]).astype(float)
    #long = (dff.iloc[row,14]).astype(float)
 # Because we only allow single row selection, the list can 
 # be converted to a row index here
  

 # Austin TX is at [30.27,-97.73]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
           center=[30.27,-97.73], zoom=10, children=[
           dl.TileLayer(id="base-layer-id"),
       # Marker with tool tip and popup
       # Column 13 and 14 define the grid-coordinates for 
       # the map
       # Column 4 defines the breed for the animal
       # Column 9 defines the name of the animal
           dl.Marker(position=[30.27,-97.73],
              children=[
              dl.Tooltip(dff.iloc[row,4]),
              dl.Popup([
                 html.H1("Animal Name"),
                html.P(dff.iloc[row,9])
         ])
      ])
   ])
]



app.run_server(debug=True)


Confirm Success
Dash app running on http://127.0.0.1:21683/
